# Data Cleaner for Books.toscrape.com ETL Project
## TTTC3213

**Data Processing Types:**
1. Price Conversion
2. Availability Extraction
3. Missing Value Handling
4. Category Standardization
5. Text Cleaning

## 1. Import Dependencies

In [ ]:
import pandas as pd
import numpy as np
import re
import os

## 2. Load Raw Data

In [ ]:
raw_data_path = os.path.join(os.getcwd(), '..', 'data', 'raw_books.csv')
raw_data_path = os.path.abspath(raw_data_path)
df_raw = pd.read_csv(raw_data_path)
print(f"Loaded {len(df_raw)} records")
df_raw.head()

## 3. Price Conversion

In [ ]:
def clean_price(price_str):
    if pd.isna(price_str):
        return np.nan
    cleaned = re.sub(r'[£$€]', '', str(price_str).strip())
    try:
        return float(cleaned)
    except ValueError:
        return np.nan

## 4. Availability Extraction

In [ ]:
def extract_stock_quantity(availability_str):
    if pd.isna(availability_str):
        return 0
    match = re.search(r'\((\d+)\s*available\)', str(availability_str))
    if match:
        return int(match.group(1))
    if 'in stock' in str(availability_str).lower():
        return 1
    return 0

## 5. Missing Value Handling

In [ ]:
def handle_missing_descriptions(df):
    missing_count = df['description'].isna().sum()
    print(f"Missing descriptions before: {missing_count}")
    df['description'] = df.apply(
        lambda row: f"No description available for '{row['title']}'." 
        if pd.isna(row['description']) or str(row['description']).strip() == '' 
        else row['description'], axis=1)
    print(f"Missing descriptions after: {df['description'].isna().sum()}")
    return df

## 6. Category Standardization

In [ ]:
def standardize_category(category_str):
    if pd.isna(category_str):
        return 'Unknown'
    cleaned = ' '.join(str(category_str).split())
    cleaned = re.sub(r'[^\w\s\-]', '', cleaned)
    cleaned = cleaned.title()
    return cleaned if cleaned else 'Unknown'

## 7. Text Cleaning

In [ ]:
def clean_text(text):
    if pd.isna(text):
        return text
    return ' '.join(str(text).split())

## 8. Category Creation Functions

In [ ]:
def create_price_category(price):
    if pd.isna(price): return 'Unknown'
    elif price < 20: return 'Budget (Under £20)'
    elif price < 35: return 'Mid-range (£20-£35)'
    elif price < 50: return 'Premium (£35-£50)'
    else: return 'Luxury (Over £50)'

def create_rating_category(rating):
    if pd.isna(rating) or rating == 0: return 'No Rating'
    elif rating <= 2: return 'Low (1-2 stars)'
    elif rating <= 3: return 'Medium (3 stars)'
    else: return 'High (4-5 stars)'

def calculate_value_score(row):
    if pd.isna(row['price_clean']) or row['price_clean'] <= 0: return 0
    if pd.isna(row['rating']) or row['rating'] == 0: return 0
    return round(row['rating'] / (row['price_clean'] / 10), 2)

## 9. Execute Cleaning Pipeline

In [ ]:
df_clean = df_raw.copy()

print("[1/5] Processing prices...")
df_clean['price_clean'] = df_clean['price'].apply(clean_price)

print("[2/5] Extracting stock quantities...")
df_clean['stock_quantity'] = df_clean['availability'].apply(extract_stock_quantity)
df_clean['in_stock'] = df_clean['stock_quantity'] > 0

print("[3/5] Handling missing values...")
df_clean = handle_missing_descriptions(df_clean)

print("[4/5] Standardizing categories...")
df_clean['category_clean'] = df_clean['category'].apply(standardize_category)

print("[5/5] Cleaning text fields...")
df_clean['title_clean'] = df_clean['title'].apply(clean_text)
df_clean['description_clean'] = df_clean['description'].apply(clean_text)

print("Creating analytical categories...")
df_clean['price_category'] = df_clean['price_clean'].apply(create_price_category)
df_clean['rating_category'] = df_clean['rating'].apply(create_rating_category)
df_clean['value_score'] = df_clean.apply(calculate_value_score, axis=1)

print("Data Cleaning Complete!")

## 10. Save Cleaned Data

In [ ]:
output_columns = ['title_clean', 'category_clean', 'price_clean', 'rating', 'rating_category',
    'price_category', 'value_score', 'stock_quantity', 'in_stock', 'description_clean',
    'upc', 'image_url', 'book_url', 'title', 'price', 'availability', 'category', 'description']
output_columns = [col for col in output_columns if col in df_clean.columns]

cleaned_data_path = os.path.join(os.getcwd(), '..', 'data', 'cleaned_books.csv')
cleaned_data_path = os.path.abspath(cleaned_data_path)
os.makedirs(os.path.dirname(cleaned_data_path), exist_ok=True)

df_clean[output_columns].to_csv(cleaned_data_path, index=False, encoding='utf-8')
print(f"Saved cleaned data to {cleaned_data_path}")

## 11. Display Sample

In [ ]:
df_clean[['title_clean', 'category_clean', 'price_clean', 'rating', 'value_score']].head(10)